<a href="https://colab.research.google.com/github/pedrodeparis/Hyperparameter-optimization/blob/main/Hyperparameter_optimization_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np

In [22]:
casa = pd.read_excel("house.xlsx")
casa.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,Gd,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,TA,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,Gd,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,TA,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,Gd,...,192,84,0,0,0,0,0,12,2008,250000


In [23]:
casa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   OverallQual    1460 non-null   int64  
 5   OverallCond    1460 non-null   int64  
 6   YearBuilt      1460 non-null   int64  
 7   YearRemodAdd   1460 non-null   int64  
 8   MasVnrArea     1452 non-null   float64
 9   ExterQual      1460 non-null   object 
 10  ExterCond      1460 non-null   object 
 11  BsmtFinSF1     1460 non-null   int64  
 12  BsmtFinSF2     1460 non-null   int64  
 13  BsmtUnfSF      1460 non-null   int64  
 14  TotalBsmtSF    1460 non-null   int64  
 15  1stFlrSF       1460 non-null   int64  
 16  2ndFlrSF       1460 non-null   int64  
 17  LowQualFinSF   1460 non-null   int64  
 18  GrLivAre

In [24]:
casa = casa.dropna()

In [25]:
casa = pd.get_dummies(casa)
casa.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,YrSold,SalePrice,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_TA
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,2008,208500,0,0,1,0,0,0,0,1
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,2007,181500,0,0,0,1,0,0,0,1
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,2008,223500,0,0,1,0,0,0,0,1
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,2006,140000,0,0,0,1,0,0,0,1
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,2008,250000,0,0,1,0,0,0,0,1


In [26]:
labels = np.array(casa["SalePrice"])

features = casa.drop("SalePrice", axis = 1)

feature_list = list(casa.columns)

casa = np.array(features)

In [27]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [28]:
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [29]:
gbr = GradientBoostingRegressor()
gbr.fit(train_features, train_labels)
gbr_pred = gbr.predict(test_features)

In [32]:
errors_gbr = abs(gbr_pred - test_labels)

r_sq = gbr.score(test_features, test_labels)
print("r^2:", r_sq)

print("MAE:", metrics.mean_absolute_error(test_labels, gbr_pred))
print("MSE:", metrics.mean_squared_error(test_labels, gbr_pred))
print("RMSE:", np.sqrt(metrics.mean_squared_error(test_labels, gbr_pred)))

r^2: 0.8936785136874313
MAE: 18508.150721846523
MSE: 863374794.1165938
RMSE: 29383.24002074301


In [36]:
from sklearn.model_selection import GridSearchCV
from tables.file import parameters


parameters = {"learning_rate" : [0.03],
              "subsample" : [0.2],
              "n_estimators" : [1000],
              "max_depth" : [8]

}

grid_search = GridSearchCV(gbr, parameters, scoring = "r2", cv=2, n_jobs = 1)

In [37]:
grid_search.fit(train_features, train_labels)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=1,
             param_grid={'learning_rate': [0.03], 'max_depth': [8],
                         'n_estimators': [1000], 'subsample': [0.2]},
             scoring='r2')

In [38]:
print("Resultados do GridSearch")
print("Melhores estimadores:", grid_search.best_estimator_)
print("Melhores paramrtros:", grid_search.best_params_)

Resultados do GridSearch
Melhores estimadores: GradientBoostingRegressor(learning_rate=0.03, max_depth=8, n_estimators=1000,
                          subsample=0.2)
Melhores paramrtros: {'learning_rate': 0.03, 'max_depth': 8, 'n_estimators': 1000, 'subsample': 0.2}


In [39]:
best_model = grid_search.best_estimator_

In [40]:
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.03,
 'loss': 'squared_error',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 0.2,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [41]:
gbr_tunned = GradientBoostingRegressor(alpha = 0.9,
 ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init = None,
 learning_rate = 0.03,
 loss = 'squared_error',
 max_depth = 8,
 max_features = None,
 max_leaf_nodes = None,
 min_impurity_decrease = 0.0,
 min_samples_leaf = 1,
 min_samples_split = 2,
 min_weight_fraction_leaf = 0.0,
 n_estimators = 1000,
 n_iter_no_change = None,
 random_state = None,
 subsample = 0.2,
 tol = 0.0001,
 validation_fraction = 0.1,
 verbose = 0,
 warm_start = False)

In [43]:
gbr_tunned.fit(train_features, train_labels)
gbr_tunned_pred = gbr_tunned.predict(test_features)

r_sq = gbr_tunned.score(test_features, test_labels)
print("R^2", r_sq)
print("MAE", metrics.mean_absolute_error(test_labels, gbr_tunned_pred))
print("MSE:", metrics.mean_squared_error(test_labels, gbr_tunned_pred))
print("RMSE:", np.sqrt(metrics.mean_squared_error(test_labels, gbr_tunned_pred)))

R^2 0.8984563494860186
MAE 18221.43708888558
MSE: 824576775.5599196
RMSE: 28715.44489573372
